## Step 1: Get the datasets of prompts and responses

In [ ]:
from datasets import Dataset, load_from_disk, load_dataset
import pandas as pd
import numpy as np 

root = "/mnt/pdata/knk25/active_pref_learning"

In [ ]:
ds = load_dataset("openbmb/UltraFeedback")
df = ds['train'].to_pandas()
df.head()

/home/azureuser/miniconda/envs/llms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,source,instruction,models,completions,correct_answers,incorrect_answers
0,evol_instruct,Can you write a C++ program that prompts the u...,"[alpaca-7b, pythia-12b, starchat, vicuna-33b]",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None]
1,evol_instruct,Suppose you are a content creator and want to ...,"[gpt-4, llama-2-13b-chat, starchat, ultralm-65b]",[{'annotations': {'helpfulness': {'Rating': '5...,[None],[None]
2,evol_instruct,"Identify the interrelated economic, political,...","[mpt-30b-chat, ultralm-13b, vicuna-33b, wizard...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None]
3,evol_instruct,How can I convert the decimal number 31 to bin...,"[alpaca-7b, mpt-30b-chat, vicuna-33b, wizardlm...",[{'annotations': {'helpfulness': {'Rating': '4...,[None],[None]
4,evol_instruct,Can you modify the C++ code provided below to ...,"[pythia-12b, llama-2-13b-chat, ultralm-13b, wi...",[{'annotations': {'helpfulness': {'Rating': '2...,[None],[None]


In [ ]:
# Process to a long data format

df['responses'] = df['completions'].apply(lambda x: [y['response'] for y in x])

def str_to_int(x):
    if x != "N/A":
        return int(x)
    else:
        return np.nan

def get_ratings(x):
    keys = ['helpfulness', 'honesty', 'instruction_following', 'truthfulness']
    return [np.array([str_to_int(y['annotations'][k]['Rating']) for k in keys]) for y in x]

df['scores'] = df['completions'].apply(get_ratings)
df = df[['instruction', 'responses', 'scores']].explode(['responses', 'scores'])
df.columns = ['prompt', 'response', 'scores']
df.reset_index(drop=True, inplace=True)
df.head()

,prompt,response,scores
0,Can you write a C++ program that prompts the u...,int main() {\n string country;\n // prom...,"[2, 1, 1, 1]"
1,Can you write a C++ program that prompts the u...,Here's a C++ program that prompts the user to ...,"[5, 5, 5, 5]"
2,Can you write a C++ program that prompts the u...,Sure! Here's a C++ program that prompts the us...,"[4, 4, 5, 3]"
3,Can you write a C++ program that prompts the u...,"Sure, here is the program using the C++11 algo...","[1, 1, 2, 1]"
4,Suppose you are a content creator and want to ...,To use GPT for generating compelling titles an...,"[5, 5, 4, 5]"


In [ ]:
# Remove duplicates by taking the average of the scores
df = df.groupby(['prompt', 'response'])['scores'].apply(lambda x: x.mean(axis=0)).reset_index()
df.head()

,prompt,response,scores
0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","[1.0, 1.0, 1.0, 2.0]"
1,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...","[2.0, 4.0, 4.0, 3.0]"
2,\n\nA federal judge on Thursday rejected effor...,"Based on the provided context, there isn't spe...","[3.0, 4.0, 5.0, 4.0]"
3,\n\nA federal judge on Thursday rejected effor...,It is difficult to determine the exact amount ...,"[4.0, 5.0, 3.0, 5.0]"
4,\n\nDo you think the right answer to the quest...,Friction is not the primary cause of energy lo...,"[4.0, 5.0, 4.0, 5.0]"


In [ ]:
# Save the dataset to disk and add a unique index

df['idx'] = df.index
ds = Dataset.from_pandas(df[['idx', 'prompt', 'response', 'scores']])
ds.save_to_disk(f"{root}/datasets/ultrafeedback/examples")

Saving the dataset (1/1 shards): 100%|██████████| 253923/253923 [00:00<00:00, 338586.17 examples/s]


## Step 2: Pair up the prompts and responses

In [ ]:
ds = load_from_disk(f"{root}/datasets/ultrafeedback/examples")
df = ds.to_pandas()
df.head()

,idx,prompt,response,scores
0,0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","[1.0, 1.0, 1.0, 2.0]"
1,1,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...","[2.0, 4.0, 4.0, 3.0]"
2,2,\n\nA federal judge on Thursday rejected effor...,"Based on the provided context, there isn't spe...","[3.0, 4.0, 5.0, 4.0]"
3,3,\n\nA federal judge on Thursday rejected effor...,It is difficult to determine the exact amount ...,"[4.0, 5.0, 3.0, 5.0]"
4,4,\n\nDo you think the right answer to the quest...,Friction is not the primary cause of energy lo...,"[4.0, 5.0, 4.0, 5.0]"


In [ ]:
# For every prompt create a list of all posible pairs of responses
def get_pairs(x):
    x = list(x)
    if len(x) < 2:
        return []
    else:
        pairs = []
        for i in range(len(x)):
            for j in range(i+1, len(x)):
                pairs.append((x[i], x[j]))
    return pairs


paired_df = df.groupby('prompt')['response'].apply(get_pairs).reset_index(name='pairs')
paired_df = paired_df.explode('pairs').dropna()
paired_df['response_a'] = paired_df['pairs'].apply(lambda x: x[0])
paired_df['response_b'] = paired_df['pairs'].apply(lambda x: x[1])
paired_df.drop(columns=['pairs'], inplace=True)
paired_df.head()

,prompt,response_a,response_b
0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","According to the text, there is no mention of ..."
0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","Based on the provided context, there isn't spe..."
0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...",It is difficult to determine the exact amount ...
0,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...","Based on the provided context, there isn't spe..."
0,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...",It is difficult to determine the exact amount ...


In [ ]:
# Merge with idx and scores
paired_df = paired_df.merge(
    df[['idx', 'prompt', 'response', 'scores']], 
    left_on=['prompt', 'response_a'], 
    right_on=['prompt', 'response'], 
    suffixes=('', '_a'),
).drop(columns=['response'])
paired_df = paired_df.merge(
    df[['idx', 'prompt', 'response', 'scores']], 
    left_on=['prompt', 'response_b'], 
    right_on=['prompt', 'response'], 
    suffixes=('', '_b')
).drop(columns=['response'])
paired_df.rename(columns={'idx': 'idx_a', 'scores': 'scores_a'}, inplace=True)
paired_df = paired_df[['prompt', 'response_a', 'response_b', 'idx_a', 'idx_b', 'scores_a', 'scores_b']]
paired_df.head()

,prompt,response_a,response_b,idx_a,idx_b,scores_a,scores_b
0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","According to the text, there is no mention of ...",0,1,"[1.0, 1.0, 1.0, 2.0]","[2.0, 4.0, 4.0, 3.0]"
1,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","Based on the provided context, there isn't spe...",0,2,"[1.0, 1.0, 1.0, 2.0]","[3.0, 4.0, 5.0, 4.0]"
2,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...",It is difficult to determine the exact amount ...,0,3,"[1.0, 1.0, 1.0, 2.0]","[4.0, 5.0, 3.0, 5.0]"
3,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...","Based on the provided context, there isn't spe...",1,2,"[2.0, 4.0, 4.0, 3.0]","[3.0, 4.0, 5.0, 4.0]"
4,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...",It is difficult to determine the exact amount ...,1,3,"[2.0, 4.0, 4.0, 3.0]","[4.0, 5.0, 3.0, 5.0]"


In [ ]:
# Save to disk
ds = Dataset.from_pandas(paired_df)
ds.save_to_disk(f"{root}/datasets/ultrafeedback/pairs")

Saving the dataset (3/3 shards): 100%|██████████| 378707/378707 [00:01<00:00, 223596.72 examples/s]


## Step 3: Concept and Preference Labels

In [ ]:
ds = load_from_disk(f"{root}/datasets/ultrafeedback/pairs")
df = ds.to_pandas()
df.head()

,prompt,response_a,response_b,idx_a,idx_b,scores_a,scores_b
0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","According to the text, there is no mention of ...",0,1,"[1.0, 1.0, 1.0, 2.0]","[2.0, 4.0, 4.0, 3.0]"
1,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","Based on the provided context, there isn't spe...",0,2,"[1.0, 1.0, 1.0, 2.0]","[3.0, 4.0, 5.0, 4.0]"
2,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...",It is difficult to determine the exact amount ...,0,3,"[1.0, 1.0, 1.0, 2.0]","[4.0, 5.0, 3.0, 5.0]"
3,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...","Based on the provided context, there isn't spe...",1,2,"[2.0, 4.0, 4.0, 3.0]","[3.0, 4.0, 5.0, 4.0]"
4,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...",It is difficult to determine the exact amount ...,1,3,"[2.0, 4.0, 4.0, 3.0]","[4.0, 5.0, 3.0, 5.0]"


In [ ]:
# Where at least one response has no concept label, set the difference to 0.0
# Rescale to [0, 1]
rel_concept_labels = np.stack((df['scores_a'] - df['scores_b']).values)
rel_concept_labels = np.where(
    np.isnan(rel_concept_labels),
    0.0,
    rel_concept_labels
)
rel_concept_labels = (rel_concept_labels - rel_concept_labels.min()) / (rel_concept_labels.max() - rel_concept_labels.min())
df['relative_concept_labels'] = list(rel_concept_labels)

In [ ]:
# Add labels based on the average score:

def reward(scores):
    scores = np.where(
        np.isnan(scores),
        3.0,
        scores
    )
    return scores.sum()

reward_a = df['scores_a'].apply(reward)
reward_b = df['scores_b'].apply(reward)

def label(reward_a, reward_b):
    if reward_a > reward_b:
        return 0
    elif reward_a < reward_b:
        return 1
    else:
        return 0.5 

df['preference_label'] = [label(a, b) for a, b in zip(reward_a, reward_b)]
df.head()

,prompt,response_a,response_b,idx_a,idx_b,scores_a,scores_b,relative_concept_labels,preference_label
0,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","According to the text, there is no mention of ...",0,1,"[1.0, 1.0, 1.0, 2.0]","[2.0, 4.0, 4.0, 3.0]","[0.375, 0.125, 0.125, 0.375]",1.0
1,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...","Based on the provided context, there isn't spe...",0,2,"[1.0, 1.0, 1.0, 2.0]","[3.0, 4.0, 5.0, 4.0]","[0.25, 0.125, 0.0, 0.25]",1.0
2,\n\nA federal judge on Thursday rejected effor...,"According to the Brookings Institution, $271.2...",It is difficult to determine the exact amount ...,0,3,"[1.0, 1.0, 1.0, 2.0]","[4.0, 5.0, 3.0, 5.0]","[0.125, 0.0, 0.25, 0.125]",1.0
3,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...","Based on the provided context, there isn't spe...",1,2,"[2.0, 4.0, 4.0, 3.0]","[3.0, 4.0, 5.0, 4.0]","[0.375, 0.5, 0.375, 0.375]",1.0
4,\n\nA federal judge on Thursday rejected effor...,"According to the text, there is no mention of ...",It is difficult to determine the exact amount ...,1,3,"[2.0, 4.0, 4.0, 3.0]","[4.0, 5.0, 3.0, 5.0]","[0.25, 0.375, 0.625, 0.25]",1.0


In [ ]:
# Save to disk
labels_df = df[['idx_a', 'idx_b', 'relative_concept_labels', 'preference_label']]
labels_ds = Dataset.from_pandas(labels_df)
labels_ds.save_to_disk(f"{root}/datasets/ultrafeedback/labels/openbmb_average")

Saving the dataset (0/1 shards):  87%|████████▋ | 329000/378707 [00:00<00:00, 3164517.93 examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 378707/378707 [00:00<00:00, 2618908.10 examples/s]


## Step 4: Generate train/val/test splits

In [ ]:
pairs = load_from_disk(f"{root}/datasets/ultrafeedback/pairs")
pairs_df = pairs.to_pandas()
print(len(pairs_df))

378707


In [ ]:
splits_df = pairs_df[['idx_a', 'idx_b']].copy()
splits_df['split'] = np.random.choice(['train', 'val', 'test'], p=[0.7, 0.1, 0.2], random_state=42)
splits_df.to_csv(f"{root}/datasets/ultrafeedback/splits.csv", index=False)